<a href="https://colab.research.google.com/github/AbigailBuchholz/test/blob/main/bach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#Hello and welcome to my attempt at solving this project! 

# The first thing I do is import all the needed info
#I usually just import everything I think of so I don't have to add anything in later
import pandas as pd
from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.linear_model import ElasticNet
#unzip your data and label it
!wget https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
!unzip bach.zip
bach=pd.read_csv('bach.csv')
#print your data to get a look at what it has
bach


--2021-03-12 23:14:03--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip.6’

bach.zip.6          100%[===================>]  40.78K  --.-KB/s    in 0.01s   

2021-03-12 23:14:03 (3.35 MB/s) - ‘bach.zip.6’ saved [41761/41761]

Archive:  bach.zip
replace bach.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [49]:

#Lets look around at the data we have
print(bach.columns)
print(bach.info)
#by using describe we see that the event number is being used. We don't really need that, so let's drop that column
bach.drop(['choral_ID','event_number'], axis = 1) 
bach.describe()


Index(['choral_ID', 'event_number', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G',
       'G#', 'A', 'A#', 'B', 'bass', 'meter', 'chord_label'],
      dtype='object')
<bound method DataFrame.info of      choral_ID  event_number    C  C#    D  ...   A#   B bass meter chord_label
0     000106b_             1  YES  NO   NO  ...   NO  NO    F     3         F_M
1     000106b_             2  YES  NO   NO  ...   NO  NO    E     5         C_M
2     000106b_             3  YES  NO   NO  ...   NO  NO    E     2         C_M
3     000106b_             4  YES  NO   NO  ...   NO  NO    F     3         F_M
4     000106b_             5  YES  NO   NO  ...   NO  NO    F     2         F_M
...        ...           ...  ...  ..  ...  ...  ...  ..  ...   ...         ...
5660  015505b_           105   NO  NO  YES  ...  YES  NO    G     4         G_m
5661  015505b_           106   NO  NO  YES  ...   NO  NO    G     3         G_m
5662  015505b_           107  YES  NO   NO  ...   NO  NO    C     5         C_M
5663 

,event_number,meter
count,5665.000000,5665.000000
mean,53.374404,3.134863
std,37.268208,1.109710
min,1.000000,1.000000
25%,24.000000,2.000000
50%,48.000000,3.000000
75%,75.000000,4.000000
max,207.000000,5.000000


In [56]:
#I know we for sure need to onehot encode the bass column, so im going to do that
one_hot=pd.get_dummies(bach['bass'])
#joing the one hot encoded dataframe to the origional
test=pd.concat([bach,one_hot],axis=1)
#now we drop the bass column from the origional dataframe
bach_hotshotted=test.drop(['bass','choral_ID','event_number'], axis=1)
bach_hotshotted.columns

#I don't know enough about music, but I feel like we should also hotshot the meter column?
#I'm going to do so because it can't hurt, right?

#we need to onehot encode the meter column
one_hot=pd.get_dummies(bach_hotshotted['meter'])
#joing the one hot encoded dataframe to the otigional
test2=pd.concat([bach_hotshotted,one_hot],axis=1)
#now we drop the bass column from the origional dataframe
finalbach=test2.drop('meter', axis=1)

finalbach

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,chord_label,A,A#,Ab,B,Bb,C,C#,D,D#,Db,E,Eb,F,F#,G,G#,1,2,3,4,5
0,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G_m,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5661,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G_m,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
5662,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C_M,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5663,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C_M,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [63]:
#the meter columns just have numbers, so im going to rename them for my own sanity
finalbach.rename(columns={1:'One Meter', 2:'Two Meter',3:'Three Meter',4:'Four Meter',5:'Five Meter'}, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B',
       'chord_label', 'A', 'A#', 'Ab', 'B', 'Bb', 'C', 'C#', 'D', 'D#', 'Db',
       'E', 'Eb', 'F', 'F#', 'G', 'G#', 'One Meter', 'Two Meter',
       'Three Meter', 'Four Meter', 'Five Meter'],
      dtype='object')

In [77]:
#It was at this point I realized I would have to onehot the whole thing
#because when I tried to predict it told me I couldnt convert floats like "yes" and "no"
#So now I will try to onehot a bunch of rows at once to save time

df=pd.get_dummies(finalbach, columns=['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B','A', 'A#', 'Ab', 'B', 'Bb', 'C', 'C#', 'D', 'D#', 'Db',
       'E', 'Eb', 'F', 'F#', 'G', 'G#'],drop_first=True)
#and it worked. Oh. My. Gosh. 
df.columns

Index(['chord_label', 'One Meter', 'Two Meter', 'Three Meter', 'Four Meter',
       'Five Meter', 'C_YES', 'C_1', 'C#_YES', 'C#_1', 'D_YES', 'D_1',
       'D#_YES', 'D#_1', 'E_YES', 'E_1', 'F_YES', 'F_1', 'F#_YES', 'F#_1',
       'G_YES', 'G_1', 'G#_YES', 'G#_1', 'A_YES', 'A_1', 'A#_YES', 'A#_1',
       'B_YES', 'B_1', 'A_YES', 'A_1', 'A#_YES', 'A#_1', 'Ab_1', 'B_YES',
       'B_1', 'Bb_1', 'C_YES', 'C_1', 'C#_YES', 'C#_1', 'D_YES', 'D_1',
       'D#_YES', 'D#_1', 'Db_1', 'E_YES', 'E_1', 'Eb_1', 'F_YES', 'F_1',
       'F#_YES', 'F#_1', 'G_YES', 'G_1', 'G#_YES', 'G#_1'],
      dtype='object')

In [106]:
#Now we train the classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')

#set up train test split
bachtrain, bachtest = train_test_split(df, test_size = 0.2, random_state=42)

bach_train_features = bachtrain[['One Meter', 'Two Meter', 'Three Meter', 'Four Meter',
       'Five Meter', 'C_YES', 'C_1', 'C#_YES', 'C#_1', 'D_YES', 'D_1',
       'D#_YES', 'D#_1', 'E_YES', 'E_1', 'F_YES', 'F_1', 'F#_YES', 'F#_1',
       'G_YES', 'G_1', 'G#_YES', 'G#_1', 'A_YES', 'A_1', 'A#_YES', 'A#_1',
       'B_YES', 'B_1', 'A_YES', 'A_1', 'A#_YES', 'A#_1', 'Ab_1', 'B_YES',
       'B_1', 'Bb_1', 'C_YES', 'C_1', 'C#_YES', 'C#_1', 'D_YES', 'D_1',
       'D#_YES', 'D#_1', 'Db_1', 'E_YES', 'E_1', 'Eb_1', 'F_YES', 'F_1',
       'F#_YES', 'F#_1', 'G_YES', 'G_1', 'G#_YES', 'G#_1']]
bach_train_labels = bachtrain['chord_label']

bach_test_features = bachtest[['One Meter', 'Two Meter', 'Three Meter', 'Four Meter',
       'Five Meter', 'C_YES', 'C_1', 'C#_YES', 'C#_1', 'D_YES', 'D_1',
       'D#_YES', 'D#_1', 'E_YES', 'E_1', 'F_YES', 'F_1', 'F#_YES', 'F#_1',
       'G_YES', 'G_1', 'G#_YES', 'G#_1', 'A_YES', 'A_1', 'A#_YES', 'A#_1',
       'B_YES', 'B_1', 'A_YES', 'A_1', 'A#_YES', 'A#_1', 'Ab_1', 'B_YES',
       'B_1', 'Bb_1', 'C_YES', 'C_1', 'C#_YES', 'C#_1', 'D_YES', 'D_1',
       'D#_YES', 'D#_1', 'Db_1', 'E_YES', 'E_1', 'Eb_1', 'F_YES', 'F_1',
       'F#_YES', 'F#_1', 'G_YES', 'G_1', 'G#_YES', 'G#_1']]
bach_test_labels = bachtest['chord_label']


In [107]:
#fit the classifier
clf.fit(bach_train_features, bach_train_labels)

#make predictions
predictions=clf.predict(bach_test_features)
#determine accuracy
accuracy_score(bach_test_labels, predictions)

0.7175639894086496

In [135]:
#I want to try and imporve my score, so lets see if building a bagging classifier helps?

from sklearn.ensemble import BaggingClassifier
clf = tree.DecisionTreeClassifier(criterion='entropy')

bagging_clf = BaggingClassifier(clf, n_estimators=500, max_samples=500, 
                                bootstrap=True, n_jobs=5)
bagging_clf.fit(bach_train_features, bach_train_labels)
predictions = bagging_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7396293027360988

In [136]:
#maybe try it without replacement?
from sklearn.ensemble import BaggingClassifier
clf = tree.DecisionTreeClassifier(criterion='entropy')

bagging_clf = BaggingClassifier(clf, n_estimators=500, max_samples=500, 
                                bootstrap=False, n_jobs=5)
bagging_clf.fit(bach_train_features, bach_train_labels)
predictions = bagging_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7378640776699029

In [161]:
#Now im going to try random subspaces, because honest why not

subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=100, 
                                bootstrap=True, n_jobs=-3)
subspace_clf.fit(bach_train_features, bach_train_labels)
predictions = subspace_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.732568402471315

In [157]:
#Lastly lets try random patches because I've already done all the other ones and it doesn't look like im going to get above 80% accuracy

In [189]:
subspace_clf = BaggingClassifier(clf, n_estimators=40, max_features=20, 
                                 max_samples=500, bootstrap=True, n_jobs=4)
subspace_clf.fit(bach_train_features, bach_train_labels)
predictions = subspace_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7131509267431597

In the end, after messing around with literally every number/classifier, my accuracy for everything averaged around 72, which could be worse!